# Assignment 1: Language modelling

In this assignment you will implement and train two or three neural language models: the fixed-window model, the recurrent neural network model from Unit&nbsp;1-2, and optionally a model based on the Transformer architecture from Unit&nbsp;1-3. You will evaluate these models by computing their perplexity on a benchmark dataset.

In [1]:
import torch

For this lab, you should use the GPU if you have one:

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    # NVIDIA
# device = torch.device('mps')    # Apple Silicon
device

device(type='cuda')

## Data

The data for this assignment is [WikiText](https://blog.salesforceairesearch.com/the-wikitext-long-term-dependency-language-modeling-dataset/), a collection of more than 100 million tokens extracted from the “Good” and “Featured” articles on Wikipedia. We will use the small version of the dataset, which contains slightly more than 2.5 million tokens.

The next cell contains code for an object that will act as a container for the “training” and the “validation” section of the data. We fill this container by reading the corresponding text files. The only processing we do is to whitespace-tokenise and to replace each newline with an end-of-sentence token.

In [3]:
UNK = "<unk>"

class WikiText(object):

    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.train = self.read_data('wiki.train.tokens')
        self.valid = self.read_data('wiki.valid.tokens')

    def read_data(self, path):
        ids = []
        with open(path, encoding='utf-8') as source:
            for line in source:
                for word in line.split() + ['<eos>']:
                    if word not in self.word2idx:
                        self.word2idx[word] = len(self.word2idx)
                        self.idx2word.append(word)
                    ids.append(self.word2idx[word])
        return ids
    
    def __len__(self):
        return len(self.idx2word)

The cell below loads the data and prints the total number of tokens and the size of the vocabulary.

In [4]:
wikitext = WikiText()
UNK_IDX = wikitext.word2idx[UNK]

print('Tokens in train:', len(wikitext.train))
print('Tokens in valid:', len(wikitext.valid))
print('Vocabulary size:', len(wikitext.word2idx))

Tokens in train: 2088628
Tokens in valid: 217646
Vocabulary size: 33278


## Problem 1: Fixed-window model

In this section, you will implement and train the fixed-window neural language model proposed by [Bengio et al. (2003)](http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf) and presented in the lectures. Recall that an input to the network takes the form of a vector of $n-1$ integers representing the preceding words. Each integer is mapped to a vector via an embedding layer. (All positions share the same embedding.) The embedding vectors are then concatenated and sent through a two-layer feed-forward network with a non-linearity in the form of a rectified linear unit (ReLU) and a final softmax layer.

### Problem 1.1: Vectorise the data

Your first task is to write code for transforming the data in the WikiText container into a vectorised form that can be fed to the fixed-window model. Concretely, you will implement a [collate function](https://pytorch.org/docs/stable/data.html#dataloader-collate-fn) in the form of a callable vectoriser object. Complete the skeleton code in the cell below:

In [5]:
class FixedWindowVectorizer(object):
    def __init__(self, n):
        self.n = n

    def __call__(self, data):

        X = []
        y = []
        for i in range(len(data) - self.n + 1):
            ngram = data[i:i+self.n]
            X.append(ngram[:-1])
            y.append(ngram[-1])
        return torch.tensor(X), torch.tensor(y)

Your code should implement the following specification:

**__init__** (*self*, *n*)

> Creates a new vectoriser with n-gram order $n$. Your code should be able to handle arbitrary n-gram orders $n \geq 1$.

**__call__** (*self*, *data*)

> Transforms WikiText *data* (a list of word ids) into a pair of tensors $\mathbf{X}$, $\mathbf{y}$ that can be used to train the fixed-window model. Let $N$ be the total number of $n$-grams from the token list; then $\mathbf{X}$ is a matrix with shape $(N, n-1)$ and $\mathbf{y}$ is a vector with length $N$.

#### 🤞 Test your code

Test your implementation by running the code in the next cell. Does the output match your expectation?

In [6]:
valid_x, valid_y = FixedWindowVectorizer(3)(wikitext.valid)

print(valid_x.shape, valid_y.shape)

torch.Size([217644, 2]) torch.Size([217644])


### Problem 1.2: Implement the model

Your next task is to implement the fixed-window model based on the graphical specification given in the lecture.

In [7]:
import torch.nn as nn

class FixedWindowModel(nn.Module):
    def __init__(self, n, n_words, embedding_dim=64, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(n_words, embedding_dim)
        self.n = n-1
        self.head = nn.Sequential(
            nn.Linear(embedding_dim * self.n, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, n_words)
        )
        

    def forward(self, x):
        batch, *_ = x.shape
        x = self.embedding(x).view(batch, -1)
        return self.head(x)
    
    def generate(self, context, n=10, temperature=1.0):
        self.eval()
        device = next(self.parameters()).device
        if len(context) < self.n:
            context = torch.cat([torch.full((self.n - len(context),), UNK_IDX, dtype=torch.long), context])
        with torch.no_grad():
            context = context[-self.n:].to(device)
            for _ in range(n):
                logits = self(context.unsqueeze(0)).squeeze(0)
                word = torch.multinomial(torch.softmax(logits / temperature, dim=-1), 1)
                context = torch.cat([context[1:], word])
                yield word

Here is the specification of the two methods:

**__init__** (*self*, *n*, *n_words*, *embedding_dim*=64, *hidden_dim*=64)

> Creates a new fixed-window neural language model. The argument *n* specifies the model&rsquo;s $n$-gram order. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the hidden layer of the feedforward network, respectively; their default value is 64.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, n-1)$, where $B$ is the batch size. The output of the forward pass is a tensor of shape $(B, V)$ where $V$ is the number of words in the vocabulary.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

### Problem 1.3: Train the model

Next, write code to train the fixed-window model using minibatch gradient descent and the cross-entropy loss function. This should be a straightforward generalisation of the training loops that you have seen so far. Complete the skeleton code in the cell below:

In [8]:
import torch.optim as optim
import torch.nn.functional as F
import tqdm


def train_fixed_window(n, n_epochs=2, batch_size=3072, lr=1e-2):
    model = FixedWindowModel(n, len(wikitext))
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    train_x, train_y = FixedWindowVectorizer(n)(wikitext.train)
    train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    valid_x, valid_y = FixedWindowVectorizer(n)(wikitext.valid)
    valid_dataset = torch.utils.data.TensorDataset(valid_x, valid_y)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


    train_losses = []
    valid_losses = []
    perplexitys = []

    with tqdm.tqdm(total = n_epochs) as pbar:
      for t in range(n_epochs):
        pbar.set_description(f'Epoch {t+1}')
        model.train()
        train_loss = 0

        for x, y in train_loader:
          x, y = x.to(device), y.to(device)
          optimizer.zero_grad()
          output = model(x)
          loss = F.cross_entropy(output, y)
          loss.backward()
          optimizer.step()
          train_loss += loss.item()
        train_loss=train_loss/len(train_loader)
        # Evaluation
        model.eval()
        valid_loss = 0
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                output_valid = model(x)
                valid_loss += F.cross_entropy(output_valid, y).cpu().item()
              
        valid_loss = valid_loss / len(valid_loader)
        perplexity = torch.exp(torch.tensor(valid_loss))
        pbar.set_postfix(perplexity=perplexity.item(), train_loss=train_loss, valid_loss=valid_loss)
        pbar.update()
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        perplexitys.append(perplexity)

    history = {'train_loss': train_losses, 'valid_loss': valid_losses, 'perplexity': perplexitys}
    
    return model, history

Here is the specification of the training function:

**train_fixed_window** (*n*, *n_epochs* = 2, *batch_size* = 3072, *lr* = 0.01)

> Trains a fixed-window neural language model of order *n* using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

The code in the cell below trains a trigram model.

In [9]:
model_fixed_window, history = train_fixed_window(3)

Epoch 2: 100%|██████████| 2/2 [00:43<00:00, 21.68s/it, perplexity=306, train_loss=5.5, valid_loss=5.72] 


#### Performance goal

Your submitted notebook must contain output demonstrating a validation perplexity of **at most 360** after training for two epochs with the default parameters.

⚠️ Computing the validation perplexity in one go (for the full validation set) will most probably exhaust your computer’s memory and/or take a lot of time. Instead, do the computation at the minibatch level and aggregate the results.

#### 🤞 Test your code

To see whether your network is learning something, print or plot the loss and/or the perplexity on the training data. If the two values do not decrease during training, try to find the problem before wasting time (and electricity) on useless computation.

Training and even evaluation will take some time – on a CPU, you should expect several minutes per epoch, depending on hardware. Our reference implementation uses a GPU and runs in 45&nbsp;seconds on a MacBook Pro (2023).

## Problem 2: Recurrent neural network model

In this section, you will implement the recurrent neural network language model. Recall that an input to this model is a vector of word ids. Each integer is mapped to an embedding vector. The sequence of embedded vectors is then fed into an unrolled LSTM. At each position $i$ in the sequence, the hidden state of the LSTM at that position is sent through a linear transformation into a final softmax layer representing the probability distribution over the words at position $i+1$. In theory, the input vector could represent the complete training data; for practical reasons, however, we will truncate the input to some fixed value *bptt_len*. This length is called the **backpropagation-through-time horizon**.

### Problem 2.1: Vectorise the data

As in the previous problem, your first task is to transform the data in the WikiText container into a vectorised form that can be fed to the model.

In [10]:
class RNNVectorizer(object):
    def __init__(self, bptt_len):
        self.bptt_len = bptt_len

    def __call__(self, data):
        X ,Y = [], []
        for idx in range(self.bptt_len, len(data) - 1):
            X.append(data[idx - self.bptt_len:idx])
            Y.append(data[idx - self.bptt_len + 1:idx + 1])
        return torch.tensor(X, dtype=torch.long), torch.tensor(Y, dtype=torch.long)

Your vectoriser should meet the following specification:

**__init__** (*self*, *bptt_len*)

> Creates a new vectoriser. The parameter *bptt_len* specifies the backpropagation-through-time horizon.

**__call__** (*self*, *data*)

> Transforms a list of token indexes *data* into a pair of tensors $\mathbf{X}$, $\mathbf{Y}$ that can be used to train the recurrent neural language model. The rows of both tensors represent contiguous subsequences of token indexes of length *bptt_len*. Compared to the sequences in $\mathbf{X}$, the corresponding sequences in $\mathbf{Y}$ are shifted one position to the right. More precisely, if the $i$th row of $\mathbf{X}$ is the sequence that starts at token position $j$, then the same row of $\mathbf{Y}$ is the sequence that starts at position $j+1$.

#### 🤞 Test your code

Test your implementation by running the following code:

In [11]:
valid_x, valid_y = RNNVectorizer(32)(wikitext.valid)

print(valid_x.size(), valid_y.size())

torch.Size([217613, 32]) torch.Size([217613, 32])


### Problem 2.2: Implement the model

Your next task is to implement the recurrent neural network model based on the graphical specification.

In [12]:
import torch.nn as nn

class RNNModel(nn.Module):
    
    def __init__(self, n_words, embedding_dim=64, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(n_words, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.head = nn.Linear(hidden_dim, n_words)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        if hidden is not None:
            x, hidden = self.lstm(x, hidden)
        else:
            x, hidden = self.lstm(x)
        return self.head(x), hidden
    
    def generate(self, context, n=10, temperature=1.0):
        device = next(self.parameters()).device
        hidden = None
        self.eval()
        with torch.no_grad():
            # build hidden state
            for x in context:
                x = x.unsqueeze(0).to(device)
                _, hidden = self(x, hidden)
            # generate next words
            for _ in range(n):
                x, hidden = self(x, hidden)
                x = x / temperature
                x = torch.multinomial(torch.softmax(x, dim=-1), 1)
                x = x.squeeze(0)
                yield x


Your implementation should follow this specification:

**__init__** (*self*, *n_words*, *embedding_dim* = 64, *hidden_dim* = 64)

> Creates a new recurrent neural network language model based on an LSTM. The argument *n_words* is the number of words in the vocabulary. The arguments *embedding_dim* and *hidden_dim* specify the dimensionalities of the embedding layer and the LSTM hidden layer, respectively; their default value is 64.

**forward** (*self*, *x*)

> Computes the network output on an input batch *x*. The shape of *x* is $(B, H)$, where $B$ is the batch size and $H$ is the length of each input sequence. The shape of the output tensor is $(B, H, V)$, where $V$ is the size of the vocabulary.

#### 🤞 Test your code

Test your code by instantiating the model and feeding it a batch of examples from the training data.

### Problem 2.3: Train the model

The training loop for the recurrent neural network model is essentially identical to the loop that you wrote for the feed-forward model. The only thing to note is that the cross-entropy loss function expects its input to be a two-dimensional tensor; you will therefore have to re-shape the output tensor from the LSTM as well as the gold-standard output tensor in a suitable way. The most efficient way to do so is to use the [`view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view) method.

In [18]:
def train_rnn(n_epochs=2, batch_size=3072, bptt_len=32, lr=1e-2):
    model = RNNModel(len(wikitext))
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_x, train_y = RNNVectorizer(bptt_len)(wikitext.train)

    train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_x, valid_y = RNNVectorizer(bptt_len)(wikitext.valid)
    valid_dataset = torch.utils.data.TensorDataset(valid_x, valid_y)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    train_losses = []
    valid_losses = []
    perplexitys = []

    with tqdm.tqdm(total = n_epochs) as pbar:
      for t in range(n_epochs):
        pbar.set_description(f'Epoch {t+1}')
        model.train()
        train_loss = 0

        for x, y in train_loader:
          x, y = x.to(device), y.to(device)
          output, _ = model(x)
          # https://discuss.pytorch.org/t/pytorch-lstm-target-dimension-in-calculating-cross-entropy-loss/30398/10
          B, C, _ = output.shape
          output = output.view(B*C, -1)
          y = y.view(B*C,)
          loss = F.cross_entropy(output, y)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          train_loss += loss.item()
        train_loss=train_loss/len(train_loader)
        # Evaluation
        model.eval()
        valid_losse = 0
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                output_valid, _ = model(x)
                B, C, _ = output_valid.shape
                output_valid = output_valid.view(B*C, -1)
                y = y.view(B*C,)
                valid_losse += F.cross_entropy(output_valid, y).cpu().item()
              
        valid_losse = valid_losse / len(valid_loader)
        perplexity = torch.exp(torch.tensor(valid_losse))
        pbar.set_postfix(perplexity=perplexity.item(), train_loss=train_loss, valid_loss=valid_losse)
        pbar.update()
        train_losses.append(train_loss)
        valid_losses.append(valid_losse)
        perplexitys.append(perplexity)

    history = {'train_loss': train_losses, 'valid_loss': valid_losses, 'perplexity': perplexitys}
    
    return model, history

Here is the specification of the training function:

**train_rnn** (*n_epochs* = 2, *batch_size* = 3072, *bptt_len* = 32, *lr* = 0.01)

> Trains a recurrent neural network language model on the WikiText data using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. The parameter *bptt_len* specifies the length of the backpropagation-through-time horizon, that is, the length of the input and output sequences. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*. After each epoch, prints the perplexity of the model on the validation data.

Evaluate your model by running the following code cell:

In [25]:
model_rnn, history = train_rnn(batch_size=3074//5, lr=1e-2/5) # if we drop the batch size, we need to drop the learning rate

Epoch 2: 100%|██████████| 2/2 [08:15<00:00, 247.79s/it, perplexity=253, train_loss=5.01, valid_loss=5.53]


#### Performance goal

Your submitted notebook must contain output demonstrating a validation perplexity of **at most 280** after training for two epochs with the default hyperparameters.

## Problem 3: Transformer model (optional)

If you are up for a challenge, try implementing a Transformer-based language model. The required vectoriser is identical to the vectoriser for the RNN model. For the model itself, you can use the Pytorch modules [`nn.TransformerEncoder`](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html) and [`nn.TransformerEncoderLayer`](https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoderLayer.html).  To represent positional information, follow the approach from the original Transformer paper and use sine and cosine functions of different frequencies ([details](https://nlp.seas.harvard.edu/2018/04/03/attention.html#positional-encoding)), or learn position-specific embeddings. Can you get a lower perplexity than for the RNN model?

In [29]:
"""
Transformer model is defined based on the pytorch tutorial on Language Modeling with nn.Transformer and torchtext.
URL: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
"""

import math

class TransformerVectorizer(object):
    def __init__(self, seq_length ):
        self.seq_length  = seq_length 

    def __call__(self, data):
        X ,Y = [], []
        for idx in range(self.seq_length, len(data) - 1):
            X.append(data[idx - self.seq_length :idx])
            Y.append(data[idx - self.seq_length  + 1:idx + 1])
        return torch.tensor(X, dtype=torch.long), torch.tensor(Y, dtype=torch.long)

class TransformerModel(nn.Module):
    def __init__(self,n_words, num_heads, num_layers, embedding_dim=64, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(n_words,embedding_dim)
        self.pos_encoder = PositionalEncoding(embedding_dim)
        encoderlayer = nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoderlayer,num_layers)
        self.linear = nn.Linear(hidden_dim, n_words)

    def forward(self, x, x_mask=None):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        if x_mask is None:
            x_mask = nn.Transformer.generate_square_subsequent_mask(len(x)).to(device)
        x = self.transformer_encoder(x,x_mask)
        x = self.linear(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self,embedding_dim, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-math.log(10000.0) / embedding_dim))
        pe = torch.zeros(max_len, 1, embedding_dim)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self,x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

def train_transformer(n_epochs=2, batch_size=3072, seq_length=32, lr=1e-2, init_func=None):
    model = TransformerModel(len(wikitext), num_heads=2, num_layers=2)
    if init_func is not None:
        init_func(model)
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_x, train_y = TransformerVectorizer(seq_length)(wikitext.train)

    train_dataset = torch.utils.data.TensorDataset(train_x, train_y)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    valid_x, valid_y = TransformerVectorizer(seq_length)(wikitext.valid)
    valid_dataset = torch.utils.data.TensorDataset(valid_x, valid_y)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    train_losses = []
    valid_losses = []
    perplexitys = []

    with tqdm.tqdm(total = n_epochs) as pbar:
      for t in range(n_epochs):
        pbar.set_description(f'Epoch {t+1}')
        model.train()
        train_loss = 0

        for x, y in train_loader:
          x, y = x.to(device), y.to(device)
          output = model(x)
          # https://discuss.pytorch.org/t/pytorch-lstm-target-dimension-in-calculating-cross-entropy-loss/30398/10
          B, C, _ = output.shape
          output = output.view(B*C, -1)
          y = y.view(B*C,)
          loss = F.cross_entropy(output, y)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          train_loss += loss.item()
        train_loss=train_loss/len(train_loader)
        # Evaluation
        model.eval()
        valid_losse = 0
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                output_valid = model(x)
                B, C, _ = output_valid.shape
                output_valid = output_valid.view(B*C, -1)
                y = y.view(B*C,)
                valid_losse += F.cross_entropy(output_valid, y).cpu().item()
              
        valid_losse = valid_losse / len(valid_loader)
        perplexity = torch.exp(torch.tensor(valid_losse))
        pbar.set_postfix(perplexity=perplexity.item(), train_loss=train_loss, valid_loss=valid_losse)
        pbar.update()
        train_losses.append(train_loss)
        valid_losses.append(valid_losse)
        perplexitys.append(perplexity)

    history = {'train_loss': train_losses, 'valid_loss': valid_losses, 'perplexity': perplexitys}
    
    return model, history

In [28]:
# Running on a smaller batch due to GPU memory issue 
model_transformer, history = train_transformer(batch_size = 3072//5, lr = 1e-2/5)

Epoch 2: 100%|██████████| 2/2 [08:37<00:00, 258.63s/it, perplexity=335, train_loss=5.07, valid_loss=5.81]


## Problem 4: Generation

In this section, you will implement a simple generation mechanism for the language models you have implemented.

Recall that one way to generate text with a language model is to repeatedly sample from the model’s output distribution, conditioning on some context. More specifically, this involves treating the softmax-normalised logits of the model as a multinomial distribution. The “creativeness” of the generation can be controlled with the temperature parameter of the softmax distribution.

To implement this recipe, we first ask you to extend each model with a `generate` method according to the following specification:

**generate** (*self*, *context*, *n_tokens* = 10, *temperature* = 1.0)

> Takes a batch of context tokens *context* and extends it by sampling *n_tokens* new tokens from the model’s output distribution, scaled with the temperature *temperature*. Returns the extended context.

In a second stage, you should implement a convenience function `generate` that allows you to easily generate text with different models, like this:

```
generate(model_fixed_window, 'i like', max_tokens=10, temperature=1.5)
```

In [24]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.0 MB/s eta 0:00:00m eta 0:00:010:0101

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def tokenize(text, lowercase=True):
    if lowercase:
        return [t.text.lower() for t in nlp.tokenizer(text)]
    else:
        return [t.text for t in nlp.tokenizer(text)]

def generate(model, text, max_tokens=3, temperature=1.0):
    context = torch.tensor([wikitext.word2idx.get(token, UNK_IDX) for token in tokenize(text)], dtype=torch.long)
    generated_tokens = []
    for token in model.generate(context, max_tokens, temperature):
        word = wikitext.idx2word[token]
        if word == '<eos>':
            break
        generated_tokens.append(word)
    return text+' '+' '.join(generated_tokens)


In [ ]:
generate(model_fixed_window, 'i like', max_tokens=10, temperature=1.0)

'i like God of War . Galveston were looking to survive ,'

In [27]:
generate(model_rnn, 'i like', max_tokens=10, temperature=1.0)

"i like <unk> 's work that do possibly elaborate starlings ."

'i like to murder Turner , but we representative not possess him'

Here is the specification of the convenience function:

**generate** (*model*, *context*, *max_tokens* = 10, *temperature* = 1.0)

> Takes a context sentence *context*, tokenises and vectorises it, and passes it to the specified *model* to generate new text. The new text consists of at most *max_tokens*, but is cut off at the first `<eos>` token. Returns the generated text (including the context).

## Problem 5: Parameter initialisation

The paper by Huang, Xiao Shi, et al., titled "Improving Transformer Optimization Through Better Initialization" presented at the International Conference on Machine Learning (ICML) in 2020, explores the potential improvements in the optimization of transformer models through enhanced initialization techniques. The authors argue that the traditional methods of initializing transformer models may not be optimal and propose alternative strategies that could potentially enhance model performance, convergence rates, and overall stability during training.

In [ ]:
class TransformerModel(nn.Module):
    def __init__(self,n_words, num_heads, num_layers, embedding_dim=64, hidden_dim=64):
        super().__init__()
        self.embedding = nn.Embedding(n_words,embedding_dim)
        self.pos_encoder = PositionalEncoding(embedding_dim)
        encoderlayer = nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoderlayer,num_layers)
        self.linear = nn.Linear(hidden_dim, n_words)

    def forward(self, x, hidden=None):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        if x_mask is None:
            x_mask = nn.Transformer.generate_square_subsequent_mask(len(x)).to(device)
        x = self.transformer_encoder(x,x_mask)
        x = self.linear(x)
        return x
    
def uniform(model, inrange = 0.1):
    model.embedding.weight.data.uniform_(-inrange, inrange)
    model.linear.bias.data.zero_()
    model.linear.weight.data.uniform_(-inrange, inrange)
    for layer in model.transformer_encoder.layers:
        layer.self_attn.out_proj.weight.data.uniform_(-inrange, inrange)
        layer.self_attn.out_proj.bias.data.zero_()
        layer.linear1.weight.data.uniform_(-inrange, inrange)
        layer.linear1.bias.data.zero_()
        layer.linear2.weight.data.uniform_(-inrange, inrange)
        layer.linear2.bias.data.zero_()

def normal(model, mean = 0, sd = 1):
    model.embedding.weight.data.normal_(mean=mean , std=sd)
    model.linear.bias.data.zero_()
    model.linear.weight.data.normal_(mean=mean , std=sd)
    for layer in model.transformer_encoder.layers:
        layer.self_attn.out_proj.weight.data.normal_(mean=mean , std=sd)
        layer.self_attn.out_proj.bias.data.zero_()
        layer.linear1.weight.data.normal_(mean=mean , std=sd)
        layer.linear1.bias.data.zero_()
        layer.linear2.weight.data.normal_(mean=mean , std=sd)
        layer.linear2.bias.data.zero_()

def xavier(model):
    nn.init.xavier_uniform_(model.embedding.weight)
    model.linear.bias.data.zero_()
    nn.init.xavier_uniform_(model.linear.weight)
    for layer in model.transformer_encoder.layers:
        nn.init.xavier_uniform_(layer.self_attn.out_proj.weight)
        layer.self_attn.out_proj.bias.data.zero_()
        nn.init.xavier_uniform_(layer.linear1.weight)
        layer.linear1.bias.data.zero_()
        nn.init.xavier_uniform_(layer.linear2.weight)
        layer.linear2.bias.data.zero_()

def kaiming(model):
    nn.init.kaiming_normal_(model.embedding.weight)
    model.linear.bias.data.zero_()
    nn.init.kaiming_normal_(model.linear.weight)
    for layer in model.transformer_encoder.layers:
        nn.init.kaiming_normal_(layer.self_attn.out_proj.weight)
        layer.self_attn.out_proj.bias.data.zero_()
        nn.init.kaiming_normal_(layer.linear1.weight)
        layer.linear1.bias.data.zero_()
        nn.init.kaiming_normal_(layer.linear2.weight)
        layer.linear2.bias.data.zero_()


In [ ]:
history1 = train_rnn(batch_size = 3072//5, lr = 1e-2/5, strategy='Uniform')
history2 = train_rnn(batch_size = 3072//5, lr = 1e-2/5, strategy='Normal')
history3 = train_rnn(batch_size = 3072//5, lr = 1e-2/5, strategy='Xavier')
history4 = train_rnn(batch_size = 3072//5, lr = 1e-2/5, strategy='He')
history5 = train_rnn(batch_size = 3072//5, lr = 1e-2/5, strategy='Huang')

print('Perplexity for Uniform Initialization:', history1['perplexity'][1].item())
print('Perplexity for Normal Initialization:', history2['perplexity'][1].item())
print('Perplexity for Xavier Initialization:', history3['perplexity'][1].item())
print('Perplexity for He Initialization:', history4['perplexity'][1].item())
print('Perplexity for Huang et al. Initialization:', history5['perplexity'][1].item())

Epoch 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [06:55<00:00, 207.69s/it, perplexity=253, train_loss=5.01, valid_loss=5.53]

Perplexity for Uniform Initialization: 258.1259765625
Perplexity for Normal Initialization: 246.63941955566406
Perplexity for Xavier Initialization: 245.9501495361328
Perplexity for He Initialization: 249.41943359375
Perplexity for Huang et al. Initialization: 252.930419921875


The error surfaces explored when training neural networks can be very complex. Because of this, it is important to choose “good” initial values for the parameters. In PyTorch, the weights of the embedding layer are initialised by sampling from the standard normal distribution $\mathcal{N}(0, 1)$. Test how changing the initialisation affects the perplexity of your language models. Find research articles that propose different initialisation strategies.

Write a short (150&nbsp;words) report about your experiments and literature search. Use the following prompts:

* What different initialisation did you try? What results did you get?
* How do your results compare to what was suggested by the research articles?
* What did you learn? How, exactly, did you learn it? Why does this learning matter?

*Answer: In our experiments, we have tried four different initialization strategies for weights in embedding layer, lstm and final layer of the RNN model. In specific, we initialised weights using Uniform distribution in range (-1,1), Normal distribution with mean=0, std=1, Xavier Initialization and He initialization. The best perplexity result was achieved by ______ with the lowest value of ____. The difference in the perplexity value of _____ and the random initialization (in problem 2) is ____ that justifies the importance of weight initialization in the neural network training.*